In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [79]:
pretrain_features = {
    "learning_rate": 0.1,
    "batch_size": 256,
    "epochs": 1000,
    "eval_size": 4*256,
    "momentum": 0.005,
    "weight_decay": 0.0001,
}

In [115]:
def load_data():
    """
    This function loads the data from the csv files and returns it as numpy arrays.

    input: None
    
    output: x_pretrain: np.ndarray, the features of the pretraining set
            y_pretrain: np.ndarray, the labels of the pretraining set
            x_train: np.ndarray, the features of the training set
            y_train: np.ndarray, the labels of the training set
            x_test: np.ndarray, the features of the test set
    """
    x_pretrain = pd.read_csv("./pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_pretrain = pd.read_csv("./pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_train = pd.read_csv("./train_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_train = pd.read_csv("./train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_test = pd.read_csv("./test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
    scaler = StandardScaler()
    x_pretrain = scaler.fit_transform(x_pretrain)
    x_train = scaler.transform(x_train)
    x_test_transed = scaler.transform(x_test)
    x_test[x_test.columns] = x_test_transed

    return x_pretrain, y_pretrain, x_train, y_train, x_test

In [116]:
x_pretrain, y_pretrain, x_train, y_train, x_test = load_data()

c:\Users\linan\.conda\envs\DL\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [82]:
class AE(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256))

        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 1000),
            nn.Sigmoid()
            )
            
        for m in self.modules():
            if isinstance(m, nn.Linear):    
                nn.init.xavier_uniform_(m.weight)
        
        
    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def encode(self, x):
        return self.encoder(x)

In [81]:
eval_size = pretrain_features["eval_size"]
batch_size = pretrain_features["batch_size"]
ae_model = AE()
ae_model.train()
ae_model.to(device)

def train_autoencoder():
    x_tr, x_val, y_tr, y_val = train_test_split(x_pretrain, y_pretrain, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    # optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_features['learning_rate'], weight_decay=pretrain_features['weight_decay'])
    optimizer = torch.optim.SGD(ae_model.parameters(), lr=pretrain_features['learning_rate'], momentum=pretrain_features['momentum'], weight_decay=pretrain_features['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = pretrain_features['epochs']
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, _] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, _] in val_loader:
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

train_autoencoder()

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: train loss: 1.247252483338322, val loss: 1.3341416716575623
Epoch 2: train loss: 1.236640982166765, val loss: 1.3240869641304016
Epoch 3: train loss: 1.2265893205323666, val loss: 1.3144888579845428
Epoch 4: train loss: 1.2169813255748108, val loss: 1.3054885566234589
Epoch 5: train loss: 1.2077651739665707, val loss: 1.2964848279953003
Epoch 6: train loss: 1.1988220776424265, val loss: 1.287909746170044
Epoch 7: train loss: 1.1899575259474438, val loss: 1.2792080640792847
Epoch 8: train loss: 1.1810770009476077, val loss: 1.270420104265213
Epoch 9: train loss: 1.1719984040856946, val loss: 1.260985791683197
Epoch 10: train loss: 1.1625497852108597, val loss: 1.251431554555893
Epoch 11: train loss: 1.1525043814215774, val loss: 1.241013616323471
Epoch 12: train loss: 1.1415919258251022, val loss: 1.2294908165931702
Epoch 13: train loss: 1.1296438334308314, val loss: 1.2170547842979431
Epoch 14: train loss: 1.1163753685472995, val loss: 1.2031743228435516
Epoch 15: train loss: 

In [87]:
class Net(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
        # and then used to extract features from the training and test data.
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)

        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)

        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
        nn.init.xavier_normal_(self.fc3.weight)


    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        # TODO: Implement the forward pass of the model, in accordance with the architecture 
        # defined in the constructor.
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x
    
    def encode(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        return x

In [84]:
pretrained_x = ae_model.encoder(torch.tensor(x_pretrain, dtype=torch.float).to(device)).detach().cpu().numpy()

In [88]:
# model declaration
nn_model = Net()
nn_model.to(device)
nn_model.train()
    
def train_nn():
    x_tr, x_val, y_tr, y_val = train_test_split(pretrained_x, y_pretrain, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    # TODO: Implement the training loop. The model should be trained on the pretraining data. Use validation set 
    # to monitor the loss.
    optimizer = torch.optim.Adam(nn_model.parameters(), lr=0.001, weight_decay=0.0001)    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 500
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, y] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            predictions = nn_model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, y] in val_loader:
            x = x.to(device)
            y = y.to(device)
            predictions = nn_model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

train_nn()

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1: train loss: 1.4721688601324971, val loss: 0.8142157047986984
Epoch 2: train loss: 0.8000914789383335, val loss: 0.8289318531751633
Epoch 3: train loss: 0.743526157951947, val loss: 0.6710024178028107
Epoch 4: train loss: 0.6867553824421939, val loss: 0.6526826620101929
Epoch 5: train loss: 0.6338142802106987, val loss: 0.5939285904169083
Epoch 6: train loss: 0.5818165031884551, val loss: 0.5477640926837921
Epoch 7: train loss: 0.531721235353625, val loss: 0.4980751648545265
Epoch 8: train loss: 0.47984679607502273, val loss: 0.4464002028107643
Epoch 9: train loss: 0.439322679183406, val loss: 0.42803681641817093
Epoch 10: train loss: 0.39083435661221827, val loss: 0.3675011023879051
Epoch 11: train loss: 0.354710235592977, val loss: 0.3383420705795288
Epoch 12: train loss: 0.31793590893312357, val loss: 0.31042467057704926
Epoch 13: train loss: 0.2851351860476179, val loss: 0.2533969134092331
Epoch 14: train loss: 0.24876254861240332, val loss: 0.2503293305635452
Epoch 15: tra

In [100]:
featured_x_train = ae_model.encoder(torch.tensor(x_train, dtype=torch.float).to(device))
featured_x_train = nn_model.encode(featured_x_train).detach().cpu().numpy()

In [101]:
from sklearn.linear_model import HuberRegressor

In [105]:
huber_reg = HuberRegressor(max_iter=10000)
huber_reg.fit(featured_x_train, y_train)
print(huber_reg.score(featured_x_train, y_train))

0.6021049886248567


In [106]:
result_file = "results-huber.csv"

In [117]:
y_pred = np.zeros(x_test.shape[0])
y_pred = huber_reg.predict(nn_model.encode(ae_model.encoder(torch.tensor(x_test.to_numpy(), dtype=torch.float).to(device))).detach().cpu().numpy())

assert y_pred.shape == (x_test.shape[0],)
y_pred = pd.DataFrame({"y": y_pred}, index=x_test.index)
y_pred.to_csv(result_file, index_label="Id")
print(f"Predictions saved to {result_file}, all done!")

Predictions saved to results-huber.csv, all done!
